In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import random
from ta.momentum import rsi
import pyodbc
import quantstats as qs
from Rolling_Functions_Final import * 

## Set Up

In [2]:
def get_SECU(content):
    Action = 0
    for line in content.split('\n'):
        if 'END' in line:
            Action = 0
        if Action == 1:
            Secu_collector.append(str(line[:6]))
            Share_collector.append(line[34:38])
        if 'TAGTAG' in line:
            Secu_collector = []
            Share_collector = []
            Action = 1
    return Secu_collector, Share_collector


def get_file(file_path):
    try:
        with open(file_path, 'r', encoding='latin-1') as file:
            content = file.read()
    except FileNotFoundError:
        print("File not found or path is incorrect.")
    except Exception as e:
        print(f"An error occurred: {e}")
    return content 

def get_stock_data(SecuCode):
        """
        :param SecuCode int/str
        
        :return Stock Data
        """
        sql1 = f"""SELECT * FROM [jydb].[dbo].SecuMain WHERE SecuCode = '{SecuCode}' and SecuCategory = '1' """
        data = Rolling.read_jydb(sql1)
        InnerCode = data['InnerCode'][0]
        if len(data)>1:
            for row in range(len(data)):
                InnerCode = data.iloc[row,:]['InnerCode']
                sql2 = f"""SELECT 
                        [InnerCode],
                        [TradingDay],
                        [NegotiableMV]
                        from LC_DIndicesForValuation
                        where InnerCode = '{InnerCode}' order by TradingDay ASC"""
                data2 = Rolling.read_jydb(sql2)
                if len(data2)> 2:
                    return data2
        sql2 = f"""SELECT  
                [InnerCode],
                [TradingDay],
                [NegotiableMV]
                from LC_DIndicesForValuation
                where InnerCode = '{InnerCode}' order by TradingDay ASC"""
        data2 = Rolling.read_jydb(sql2)
        return data2

def final_mkt_cap_calculator(Secu_collector, Share_collector):
    mkt_cap_cal = pd.DataFrame()
    newest = []
    daybefore = []
    for i in Secu_collector:
        image = get_stock_PRICE(i)
        # print(f'The last date for STOCK {i} is at {image.loc[:,"TradingDay"][len(image)-1]}')
        newest_price = image.loc[:,'ClosePrice'][len(image)-1]
        daybefore_price = image.loc[:,'ClosePrice'][len(image)-2]
        # print(rate_of_change)
        newest.append(newest_price)
        daybefore.append(daybefore_price)

    Share_collector = [int(x) for x in Share_collector]

    mkt_cap_cal["newest_price"] = newest
    mkt_cap_cal["daybefore"] = daybefore
    mkt_cap_cal["Share"] = Share_collector
    mkt_cap_cal["mkt_change_new"] = (mkt_cap_cal["newest_price"]  * mkt_cap_cal["Share"])
    mkt_cap_cal["mkt_change_old"] = (mkt_cap_cal["daybefore"]  * mkt_cap_cal["Share"])

    final_result = mkt_cap_cal['mkt_change_new'].sum()/mkt_cap_cal['mkt_change_old'].sum()
    return final_result, mkt_cap_cal

def mkt_cap_calculator(Secu_collector):
    mkt_cap_cal = pd.DataFrame()
    for i in Secu_collector:
        image = get_stock_data(i)
        print(f'The last date for STOCK {i} is at {image.loc[:,"TradingDay"][len(image)-1]}')
        rate_of_change = image.loc[:,'NegotiableMV'][len(image)-1]/image.loc[:,'NegotiableMV'][len(image)-2]
        # print(rate_of_change)
        mkt_cap_cal[f'{i}_mktcap_change'] = [rate_of_change]
    return mkt_cap_cal

def get_stock_PRICE(SecuCode):
        """
        :param SecuCode int/str
        
        :return Stock Data
        """
        sql1 = f"""SELECT * FROM [jydb].[dbo].SecuMain WHERE SecuCode = '{SecuCode}' and SecuCategory = '1' """
        data = Rolling.read_jydb(sql1)
        InnerCode = data['InnerCode'][0]
        if len(data)>1:
            for row in range(len(data)):
                InnerCode = data.iloc[row,:]['InnerCode']
                sql2 = f"""SELECT 
                        [InnerCode],
                        [TradingDay],
                        [ClosePrice]
                        from QT_PerformanceData
                        where InnerCode = '{InnerCode}' order by TradingDay ASC"""
                data2 = Rolling.read_jydb(sql2)
                if len(data2)> 2:
                    return data2
        sql2 = f"""SELECT  
                [InnerCode],
                [TradingDay],
                [ClosePrice]
                from QT_PerformanceData
                where InnerCode = '{InnerCode}' order by TradingDay ASC"""
        data2 = Rolling.read_jydb(sql2)
        return data2

## -

## - 

## - 

## Get Stock NegotiableMV

In [3]:
file_path = r'C:\Users\ps\PycharmProjects\pythonProject\51289007202.ETF'

content = get_file(file_path)

Secu_collector, Share_collector = get_SECU(content)

mkt_cap_cal = mkt_cap_calculator(Secu_collector)

The last date for STOCK 000401 is at 2023-07-26 00:00:00
The last date for STOCK 000581 is at 2023-07-26 00:00:00
The last date for STOCK 000651 is at 2023-07-26 00:00:00
The last date for STOCK 000717 is at 2023-07-26 00:00:00
The last date for STOCK 000789 is at 2023-07-26 00:00:00
The last date for STOCK 000830 is at 2023-07-26 00:00:00
The last date for STOCK 000932 is at 2023-07-26 00:00:00
The last date for STOCK 002003 is at 2023-07-26 00:00:00
The last date for STOCK 002443 is at 2023-07-26 00:00:00
The last date for STOCK 002818 is at 2023-07-26 00:00:00
The last date for STOCK 600015 is at 2023-07-26 00:00:00
The last date for STOCK 600019 is at 2023-07-26 00:00:00
The last date for STOCK 600028 is at 2023-07-26 00:00:00
The last date for STOCK 600057 is at 2023-07-26 00:00:00
The last date for STOCK 600064 is at 2023-07-26 00:00:00
The last date for STOCK 600170 is at 2023-07-26 00:00:00
The last date for STOCK 600177 is at 2023-07-26 00:00:00
The last date for STOCK 600188 

In [4]:
mkt_cap_cal

,000401_mktcap_change,000581_mktcap_change,000651_mktcap_change,000717_mktcap_change,000789_mktcap_change,000830_mktcap_change,000932_mktcap_change,002003_mktcap_change,002443_mktcap_change,002818_mktcap_change,...,601398_mktcap_change,601577_mktcap_change,601658_mktcap_change,601668_mktcap_change,601838_mktcap_change,601857_mktcap_change,601928_mktcap_change,601939_mktcap_change,601988_mktcap_change,601998_mktcap_change
0,1.019455,0.97709,1.004134,1.02069,1.019441,1.009569,1.010889,1.012245,1.010189,1.008214,...,1.002165,1.00507,0.991935,1.001718,0.946244,0.978829,0.979574,0.993311,0.994751,0.981324


## - 

## - 

## -

## Get Stock Price

In [5]:
file_path = r'C:\Users\ps\PycharmProjects\pythonProject\51289007202.ETF'

content = get_file(file_path)

Secu_collector, Share_collector = get_SECU(content)

final_result, mkt_cap_cal = final_mkt_cap_calculator(Secu_collector, Share_collector)
final_result


0.9972317360034798

In [8]:
# get_stock_PRICE("000401")

In [7]:
1 - final_result

0.0027682639965201927

In [9]:
mkt_cap_cal

,newest_price,daybefore,Share,mkt_change_new,mkt_change_old
0,7.86,7.71,1900,14934.0,14649.0
1,17.06,17.46,1200,20472.0,20952.0
2,38.86,38.70,700,27202.0,27090.0
3,2.96,2.90,4400,13024.0,12760.0
4,8.39,8.23,2000,16780.0,16460.0
5,10.55,10.45,1800,18990.0,18810.0
6,5.57,5.51,2600,14482.0,14326.0
7,9.92,9.80,1300,12896.0,12740.0
8,6.94,6.87,2400,16656.0,16488.0
9,14.73,14.61,1700,25041.0,24837.0


In [10]:
mkt_cap_cal['mkt_change_old'].sum()

947164.0

In [11]:
mkt_cap_cal['mkt_change_new'].sum()

944542.0